# Большая домашняя работа

Эта домашнаяя работа будет уже более творческой.

Будем решать еще одно соревнование на Kaggle, с похожей задачей про предсказание кликов по рекламе: https://www.kaggle.com/c/outbrain-click-prediction

Оно уже завершилось, но мы будем делать приватные посылки для скоринга на тестовой выборке. 

Вам необходимо решить указанную задачу. Есть небольшие ограничения:
* Сбор датасета для обучения необходимо производить средствами Spark
* Нельзя обучать модели из sklearn (другие функции оттуда брать можно, например для оценки качества)

В остальном вы более-менее свободны в выборе инструментов и подходов к решению.

В качестве сдачи вам необходимо прислать ноутбук, в котором будет:
* Код на Spark в котором вы собираете датасет
* Код обучения вашей модели
* Скриншот из Kaggle, где видна ваша посылка и метрика на ней


**Важно** В задании есть лик в данных (https://www.kaggle.com/code/its7171/leakage-solution дает +0.015), его использовать нельзя!

## Критерии оценивания

Будем смотреть на MAP посылки, полученный на тестовой выборке соревнования:
* `< 0.60` (это AUC <=0.73) - до 4 баллов
* `>= 0.62` - 5
* `>= 0.64` - 6
* `>= 0.66` - 7
* `>= 0.67` (AUC примерно 0.75) - 8
* `>= 0.671` - 9
* `>= 0.672` - 10

Помимо метрики будем еще смотреть на само решение. 
* Если решение написано слишком неоптимально, то можно потерять балл (но не больше одного). 
* Если у нас будут большие сомнения по поводу того, можно ли было получить такое качество на той модели, которую вы выбрали и обучили, то можно потерять балл (тут уже много можно потерять, вплоть до обнуления работы, если найдем прям списывание).

В связи с этим есть еще один нюанс - ваше финальное решение должно получаться запуском всего вашего ноутбука при помощи “Run All Cells” на свежем кластере.


### Что делать с самым тяжелым датасетом из задания
Как вы могли заметить, в данных этого соревнования есть "тяжелый" датасет (весит порядка 30Гб в архиве), в то время как все остальные части данных в сореве весят разумно меньше.

Чтобы упростить вам жизнь с его перегонкой на кластер, мы предоработали его за вас и сорабли в архив в уже в формате паркета. Что вам нужно сделать:
* Скачиваете архив с данными `page_views` по нашей прямой ссылке с яндекс диска сразу на мастер ноду
```bash
%%writefile download_archive.sh

pip3 install wldhx.yadisk-direct

URL=$(yadisk-direct https://disk.yandex.ru/d/wcQYvAPZHpih9Q)

wget -O page_views_parquet.zip "${URL}"
```
собранный скрипт запустить в терминале `. ./download_archive.sh`, чтобы пошла скачка архива. Такой вариант скачки займет порядка **10мин**

Дальше вы его распакуете (через тот же 7z), получите директорию с паркет файлами, это займет порядка **50мин**.

Далее размещаете её на hdfs через тот же `put`

```bash
!hdfs dfs -mkdir /user/page_views

!hdfs dfs -put page_views_parquet/* /user/page_views
```
Эта операция у вас займет **60мин**, через отдельный терминал можно проверять сколько файлов перенеслось в папку на hdfs к определенному времени. 

Итого: это самый оптимальный способ закинуть табличку в паркет формате на кластер за 2ч. Пожалуйста, воспользутесь им, если будете использовать эти данные для решения задачи.

Со всеми остальными датасетами из соревнования вы вполне справитесь сами, они весят сильно меньше.

### Заметки на полях

**1.** Сохраняйте промежуточные результаты. За один присест вы домашку не решите, поэтому почаще сохраняйте свои результаты в промежуточные таблицы на кластере. Чтобы вам не было мучительно больно за бесцельно прожитые часы ожидания работы кластера.

**2.** Конвертируйте данные в Parquet и используйте по максимуму Dataframe API. Вам придется исследовать данные и проверять гипотезы, поэтому используйте быстрые инструменты для этого. С самым большим датасетом из задания мы вам уже частично помогли.

**3.** Перед тем как запускать большой запрос, проверьте вашу идею на небольшом семпле, если это возможно. Это спасет сам кучу часов.

**4.** Тюнингом параметров VW можно выжать AUC 0.732 из очень простых признаков. Для более крутых фичей советую почитать отчет одного участника конкурса: http://dsnotes.com/post/2017-01-27-lessons-learned-from-outbrain-click-prediction-kaggle-competition/. И продолжение этого топика: https://dsnotes.com/2017-02-07-large-data-feature-hashing-and-online-learning-part-2/. **Очень настоятельная рекомендация** ознакомиться с этим отчетом по данным. Он даст вам стартовые гипотезы по фичам, которые так или иначе дадут вам нормальный результат в базе для обучения модели.

На "тупых" фичах у них AUC 0.737, MAP 0.654 (они добавили еще interactions длины 2), на умных AUC 0.752, MAP 0.673, ориентируйтесь на них, они попали в топ 50. У них кстати довольно простые фичи по page_views, у них там весь код про то, как это на одной машине посчитать, у вас таких проблем нет.

**5.** **Изучите доку к vw**, посмотрите какой у него есть функционал, который мы еще не использовали на семинарах. Возможно какие-то методы работы с данными из статьи выше могут быть использованы в успешной связке с vw.

**6.** За один день домашка не делается, на хорошую оценку и за 2 не делается. За 3 - уже возможно. Пожалуйста, за опр. кол-во дней до дедлайна займитесь хотя бы размещением данных в хранилище, с этой точки уже можно будет начать делать полезную часть работы.